Best submission so far is ensemble with LinearSVC (not SVM), no max features, no KNN. (KNN hasn't been grid searched yet).

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
%matplotlib

Using matplotlib backend: MacOSX


In [3]:
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
%matplotlib inline
sns.set_palette('colorblind')
sns.set_style('white')

In [4]:
import pandas as pd
import nltk
import string

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.cross_validation import StratifiedShuffleSplit, train_test_split
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix 

In [21]:
from collections import Counter
from sklearn.pipeline import Pipeline

from sklearn.grid_search import GridSearchCV

In [7]:
def get_data():
    df = pd.read_csv('newtrain.csv')
    return df

In [8]:
def get_test_data():
    df = pd.read_csv('newtest.csv')
    return df

In [9]:
def add_features(df):
    #tokenize text
    pattern = r'''(?x)    # set flag to allow verbose regexps
         ([A-Z]\.)+        # abbreviations, e.g. U.S.A.
       | \w+([-']\w+)*        # words with optional internal hyphens
       | \$?\d+(\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
       | [!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+  # these are separate tokens (string.punctuation)
     '''
    tokenize = lambda text: nltk.regexp_tokenize(text, pattern)
    df['Tokens'] = df['Text'].apply(tokenize)
    
    def pos_tag(text):
        tuples = nltk.pos_tag(text)
        tags = []
        for t in tuples:
            tags.append(t[1])
        return tags
    
    def get_nouns(text):
        wnlemmatizer = nltk.WordNetLemmatizer()
        tuples = nltk.pos_tag(text)
        tags = []
        for t in tuples:
            if t[1][0] == 'N':
                tags.append(wnlemmatizer.lemmatize(t[0]))
        return tags
    df['Nouns'] = df['Tokens'].apply(get_nouns)
    df['POS'] = df['Tokens'].apply(pos_tag)
    make_string = lambda a: ' '.join(i for i in a)
    df['Nouns'] = df['Nouns'].apply(make_string)
    df['POS'] = df['POS'].apply(make_string)
    
    df['TokenString'] = df['Tokens'].apply(make_string)
    
    df['TokensNouns'] = df[['TokenString', 'Nouns']].apply(lambda x: ' '.join(x), axis=1)
    
    return df

In [16]:
#vectorize data
#turns words into a list of vectors - vector length is the total number of words
#Vector elements correspond to 1 word (1/0 if word is/not present in the current item)
def featurize(df, test_df, count_vec):
    pos_vec = CountVectorizer()
    vec_tar = LabelEncoder()
    targets = df['Category']
        
    counts = count_vec.fit_transform(df['TokensNouns'])
    nouns = noun_vec.fit_transform(df['Nouns'])
    pos = pos_vec.fit_transform(df['POS'])
    
    
    tfidf_transformer = TfidfTransformer()
    tfidf = tfidf_transformer.fit_transform(counts)
    
    
    test = count_vec.transform(test_df['TokensNouns'])
    test = tfidf_transformer.transform(test)
    test_nouns = noun_vec.transform(test_df['Nouns'])
    test_pos = pos_vec.transform(test_df['POS'])
    
    return tfidf, test

In [57]:
# Takes in training data, training labels, predicts labels for xtest using three methods.
def ensemble_predictor(df, test_df): 
    mod1 = LinearSVC(C=1) #vec analyzer = word, ngram(1,1)
    mod2 = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
    mod3 = LogisticRegression(C=1000)#vec analyzer = word, ngram(1,3)
    mod4 = KNeighborsClassifier(n_neighbors=10)
    mod5 = SVC(C=1000, gamma=.01)
    
    ytrain = df['Category']
    xtrain1, xtest1 = featurize(df, test_df, CountVectorizer(analyzer='word', ngram_range=(1,1)))
    #xtrain2, xtest2 = featurize(df, test_df, CountVectorizer(analyzer='word', ngram_range=(1,1)))
    xtrain3, xtest3 = featurize(df, test_df, CountVectorizer(analyzer='word', ngram_range=(1,3)))
    xtrain4, xtest4 = featurize(df, test_df, CountVectorizer(analyzer='char_wb', ngram_range=(5, 5)))
    xtrain5, xtest5 = featurize(df, test_df, CountVectorizer(analyzer='word', ngram_range=(1, 3)))
    
    
    mod1.fit(xtrain1, ytrain)
    #mod2.fit(xtrain2, ytrain)
    mod3.fit(xtrain3, ytrain)
    mod4.fit(xtrain4, ytrain)
    mod5.fit(xtrain5, ytrain)
    
    y1 = mod1.predict(xtest1)
    #y2 = mod2.predict(xtest2)
    y3 = mod3.predict(xtest3)
    y4 = mod4.predict(xtest4)
    y5 = mod5.predict(xtest5)
    
    #votes = zip(y1, y2, y3, y4, y5)
    votes = zip(y1,y3,y4,y5)
    ypredicted = []
    count = 0
    for v in votes:
        v = list(v)
        #shuffle(v)
        p = max(set(v), key=v.count)
        if len(np.unique(v)) == len(v):
            count += 1
            p = v[1]
        ypredicted.append(p)
    print("count: " + str(count))
    print("total number of predictions: " + str(len(ypredicted)))
    return ypredicted

In [43]:
#evaluate using 10 fold cross-validation, inspect results
#KNN WITH CV#
def cross_validate(targets, df):
    
    #LinearSVC
    mod = LinearSVC(C=1)
    cv = StratifiedShuffleSplit(targets, n_iter=10, test_size=.1)
    
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='word', ngram_range=(1,1)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))

    print('\nLinear SVC\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    
    
    #SVC
    mod = SVC(C=1000, gamma=0.01)
    cv = StratifiedShuffleSplit(targets, n_iter=10, test_size=.1)
    
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='word', ngram_range=(1, 3)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))

    print('\nRBF SVC\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    #naive bayes
    mod = MultinomialNB(alpha=1.0, fit_prior=True)
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='word', ngram_range=(1,1)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))
    print('\nNaive Bayes\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    mod = LogisticRegression(C=1000)
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='word', ngram_range=(1,3)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))
    print('\nLogReg\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    mod = KNeighborsClassifier(n_neighbors=10)
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='char_wb', ngram_range=(5, 5)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))
    print('\nKNN\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        predictions = ensemble_predictor(train, test)
        scores.append(accuracy_score(targets[tt], predictions))
    print('\nEnsemble Predictor\n\t mean score: {0}'.format(np.mean(scores)))
    


In [11]:
def plot_confusion_matrix(cm, title, target_names, cmap=plt.cm.coolwarm):
    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [12]:
def examine_classification(model):
    result = get_labels_features(df, 1000)
    y = result[0]
    x = result[1]
    xtrain, xtest, ytrain, ytest = train_test_split(x, labels, test_size=0.33, random_state=99)
    
    model.fit(xtrain, ytrain)
    ypredicted = model.predict(xtest)
    print(accuracy_score(ytest, ypredicted))
    print()
    print(pd.crosstab(ytest, ypredicted, 
            rownames=['True'], colnames=['Predicted'], 
            margins=True))
    print()
    #cm = confusion_matrix(ytest, ypredicted)
    #plot_confusion_matrix(cm, "Confusion Matrix", range(1,8))

In [14]:
df = get_data()
df = add_features(df)
df.head()

,Category,Text,Tokens,Nouns,POS,TokenString,TokensNouns
0,5,why are yawns contagious? when people yawn,"[why, are, yawns, contagious, when, people, yawn]",yawn people,WRB VBP NNS JJ WRB NNS VBP,why are yawns contagious when people yawn,why are yawns contagious when people yawn yawn...
1,6,what is trans fat? how to reduce that? i heard...,"[what, is, trans, fat, how, to, reduce, that, ...",trans tras body food,WP VBZ NNS JJ WRB TO VB IN PRP VBP IN NNS VBP ...,what is trans fat how to reduce that i heard t...,what is trans fat how to reduce that i heard t...
2,1,roth ira vs 401k? what is the difference betwe...,"[roth, ira, vs, 401k, what, is, the, differenc...",roth ira v difference roth ira prefer,NN NN NNS CD WP VBZ DT NN IN NN NN CC CD WRB M...,roth ira vs 401k what is the difference betwee...,roth ira vs 401k what is the difference betwee...
3,1,how many planes fedex has? i heard that it is ...,"[how, many, planes, fedex, has, i, heard, that...",plane fedex airline world,WRB JJ NNS NN VBZ PRP VBP IN PRP VBZ DT JJS NN...,how many planes fedex has i heard that it is t...,how many planes fedex has i heard that it is t...
4,2,what is the best photo slideshow creation appl...,"[what, is, the, best, photo, slideshow, creati...",photo slideshow creation application photo sli...,WP VBZ DT JJS NN NN NN NN WP VBZ DT JJS NN NN ...,what is the best photo slideshow creation appl...,what is the best photo slideshow creation appl...


In [58]:
cross_validate(df['Category'], df)


Linear SVC
	 mean score: 0.5914814814814815

RBF SVC
	 mean score: 0.5722222222222222

Naive Bayes
	 mean score: 0.4159259259259259

LogReg
	 mean score: 0.552962962962963

KNN
	 mean score: 0.5296296296296296
count: 3
total number of predictions: 270
count: 0
total number of predictions: 270
count: 2
total number of predictions: 270
count: 3
total number of predictions: 270
count: 1
total number of predictions: 270
count: 2
total number of predictions: 270
count: 1
total number of predictions: 270
count: 1
total number of predictions: 270
count: 2
total number of predictions: 270
count: 1
total number of predictions: 270

Ensemble Predictor
	 mean score: 0.5844444444444444


In [37]:
pipeline, parameters = [dict() for i in range(2)]
#set up cross validation folds
cv = StratifiedShuffleSplit(df.Category, n_iter=5, test_size=.2)

#analysis pipeline with linear svc
pipeline['svc'] = Pipeline([
    ('vect', CountVectorizer(stop_words = ENGLISH_STOP_WORDS)),
    ('tfidf', TfidfTransformer(use_idf = True)),
    ('clf', LinearSVC())])

parameters['svc'] = {
    'vect__ngram_range': ((1, 1), (1,3), (5, 5)),
    'vect__analyzer' : ('char_wb','word'),
    'clf__C': (1, 1e3, 1e-3)}

#analysis pipeline with svm with rbf kernel
pipeline['svm'] = Pipeline([
    ('vect', CountVectorizer(stop_words = ENGLISH_STOP_WORDS)),
    ('tfidf', TfidfTransformer(use_idf = True)),
    ('clf', SVC())])

parameters['svm'] = {
    'vect__ngram_range': ((1, 1), (1,3), (5, 5)),
    'vect__analyzer' : ('char_wb','word'),
    'clf__C': (1, 1e3, 1e-3),
    'clf__gamma': (1, 1e2, 1e-2)}

#analysis pipeline with logistic regression
pipeline['log'] = Pipeline([
    ('vect', CountVectorizer(stop_words = ENGLISH_STOP_WORDS)),
    ('tfidf', TfidfTransformer(use_idf = True)),
    ('clf', LogisticRegression(penalty = 'l2', solver = 'lbfgs', multi_class = 'multinomial'))])
        
parameters['log'] = {
    'vect__ngram_range': ((1, 1), (1,2), (1,3), (5, 5)),
    'vect__analyzer' : ('char_wb','word'),
    'clf__C': (1, 1e3, 1e-3)}

#analysis pipeline with naive bayes
pipeline['nb'] = Pipeline([
    ('vect', CountVectorizer(stop_words = ENGLISH_STOP_WORDS)),
    ('tfidf', TfidfTransformer(use_idf = True)),
    ('clf', MultinomialNB(fit_prior = True))])
parameters['nb'] = {
    'vect__ngram_range': ((1, 1), (1,2), (1,3), (5, 5)),
    'vect__analyzer' : ('char_wb','word')}

#analysis pipeline with kmeans
pipeline['knn'] = Pipeline([
    ('vect', CountVectorizer(stop_words = ENGLISH_STOP_WORDS)),
    ('tfidf', TfidfTransformer(use_idf = True)),
    ('clf', KNeighborsClassifier())])
parameters['knn'] = {
    'vect__ngram_range': ((1, 1), (1,2), (1,3), (5, 5)),
    'vect__analyzer' : ('char_wb','word'),
    'clf__n_neighbors': (8, 9, 10, 15),
    'clf__weights': ('uniform', 'distance')}


#fit grid search instance
for m in ['svc','svm']: #,'log','nb', 'knn']:
#for m in ['knn']:
    scorer = 'accuracy'
    gs_clf = GridSearchCV(pipeline[m], parameters[m], verbose = 1, cv = cv, n_jobs = -1, scoring = scorer)

    gs_clf = gs_clf.fit(df['TokensNouns'], df.Category)

    best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1]) #find best params

    print ('{0}:\n\tbest score: {1}\nparameters: {2}'.format(m, score, best_parameters))

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done  84 out of  90 | elapsed:   35.8s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   36.3s finished


svc:
	best score: 0.5729128014842301
parameters: {'vect__ngram_range': (1, 1), 'vect__analyzer': 'word', 'clf__C': 1}
Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 200 jobs       | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 264 out of 270 | elapsed:  5.5min remaining:    7.5s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  5.5min finished


svm:
	best score: 0.5692022263450834
parameters: {'vect__ngram_range': (1, 3), 'clf__gamma': 0.01, 'vect__analyzer': 'word', 'clf__C': 1000.0}


In [45]:
test_df = get_test_data()
test_df = add_features(test_df)

In [59]:
predictions = ensemble_predictor(df, test_df)

count: 12
total number of predictions: 1874


In [60]:
import csv
with open('submission8.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['ID', 'Category'])
    for i, p in enumerate(predictions):
        writer.writerow([i + 1, p])